In [47]:
#IPython magic to reload libraries
%load_ext autoreload
%autoreload 2

# Extra code so Juypter doesn't crash on mac os
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import sys
sys.path.insert(0, 'utils')  # noqa

import numpy as np
import gym
import ntm
import interfaces

In [49]:
# Training Params
train_iter = 1
batch_size = 1


In [50]:
# Task
env = gym.make('Copy-v0')
env = env.unwrapped
print(env.action_space)
print(env.observation_space)


Tuple(Discrete(2), Discrete(2), Discrete(5))
Discrete(6)


In [269]:
# Controller 
Controller = ntm.NTM()

Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tape_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
memory_input (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_29 (Concatenate)    (None, 11)           0           tape_input[0][0]                 
                                                                 memory_input[0][0]               
__________________________________________________________________________________________________
dense_51 (Dense)                (None, 32)           384         concatenate_29[0][0]      

In [335]:
Controller.run(tape_input=1, output_target=1)

tf.Tensor([[0.0228377  0.9309435  0.01390508 0.01920357 0.01311021]], shape=(1, 5), dtype=float32)
Loss:  [0.0715567]
(1, 2)


(<tf.Tensor: id=40014, shape=(1, 1), dtype=int64, numpy=array([[1]])>,
 <tf.Tensor: id=40017, shape=(1, 1), dtype=int64, numpy=array([[1]])>,
 <tf.Tensor: id=40020, shape=(1, 1), dtype=int64, numpy=array([[1]])>)

In [352]:
for i in range(train_iter):
    
    # Collect experince using current policy/controller
    for j in range(batch_size):
        
        episode_steps = 0
        observation = env.reset()
        env.render()

        while True:
            episode_steps += 1
            
            target = env.target[env.write_head_position]
            outputs = Controller.run(tape_input=observation, output_target=target)

            output_content = outputs[0] # Which character to write (ignored if the above sub-action is 0) 
            output_control = outputs[1] #  Whether to write to the output tape ["no","yes"] = [0,1]
            input_control = outputs[2] # Direction to move the read head ["left","right"] = [0,1]

            actions = (input_control, output_control, output_content)
            print(actions)
            observation, reward, done, info = env.step(actions)
            env.render()

            if done:
                done = False
                break
        

Total length of input instance: 4, step: 0
Observation Tape    :   ABEC  
Output Tape         :   
Targets             :   ABEC  






Loss:  [2.2568672]
(1, 2)
(1, 1, 1)
Total length of input instance: 4, step: 1
Observation Tape    :   ABEC  
Output Tape         :   B
Targets             :   ABEC  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: B)
